In [1]:
import numpy
from keras.models import load_model

In [21]:
seg_max_min={9: [136.44, 69.0],
 24: [58.14, 29.0],
 256: [90.0, 31.0],
 267: [134.46, 63.0],
 268: [102.06, 51.0],
 277: [136.44, 64.0]}

In [18]:
def get_input_output(target_seg):
    # target_seg=[24, 256, 267, 268]
    target_seg_len=len(target_seg)
    # target_seg_len=len(target_seg)
    train_series=[train[seg][0] for seg in target_seg]
    test_series=[test[seg][0] for seg in target_seg]
    validate_series=[validate[seg][0] for seg in target_seg]
    train_series_out=[train[seg][2] for seg in target_seg]
    test_series_out=[test[seg][2] for seg in target_seg]
    validate_series_out=[validate[seg][2] for seg in target_seg]
    train_series_out=numpy.array(train_series_out)
    test_series_out=numpy.array(test_series_out)
    validate_series_out=numpy.array(validate_series_out)
    train_series=numpy.array(train_series)
    test_series=numpy.array(test_series)
    validate_series=numpy.array(validate_series)
    train_data=train_series[0]
    for i in range(1,target_seg_len):
        train_data=numpy.concatenate((train_data,train_series[i]),axis=0)
    test_data=test_series[0]
    for i in range(1,target_seg_len):
        test_data=numpy.concatenate((test_data,test_series[i]),axis=0)
    validate_data=validate_series[0]
    for i in range(1,target_seg_len):
        validate_data=numpy.concatenate((validate_data,validate_series[i]),axis=0)
    train_data_out=train_series_out[0]
    for i in range(1,target_seg_len):
        train_data_out=numpy.concatenate((train_data_out,train_series_out[i]),axis=0)
    test_data_out=test_series_out[0]
    for i in range(1,target_seg_len):
        test_data_out=numpy.concatenate((test_data_out,test_series_out[i]),axis=0)
    validate_data_out=validate_series_out[0]
    for i in range(1,target_seg_len):
        validate_data_out=numpy.concatenate((validate_data_out,validate_series_out[i]),axis=0)
        
    train_data_out=train_data_out.reshape(train_data_out.shape[0],1)
    test_data_out=test_data_out.reshape(test_data_out.shape[0],1)

    model_in=[train_data]
    model_out=[train_data_out]
    test_model_in=[test_data]
    test_model_out=[test_data_out]
    validate_model_in=[validate_data]
    validate_model_out=[validate_data_out]
    return (model_in,model_out,test_model_in,test_model_out,validate_model_in,validate_model_out)

In [15]:

c=load_model(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2_0\NN\nb_nn_c.h5',compile=False)
c0=load_model(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2_0\NN\nb_nn_c0_1.h5',compile=False)
c1=load_model(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2_0\NN\nb_nn_c1.h5',compile=False)

In [16]:
# ../input/deep-pf-1/test.npy

train=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2\train.npy',allow_pickle=True).item()
test=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2\test.npy',allow_pickle=True).item()
validate=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2\validation.npy',allow_pickle=True).item()

dw=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\day_of_week.npy',allow_pickle=True).item()
ts=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\encoded_time_stamp.npy')

In [19]:
targets=[9,24,256, 267, 268, 277]
model_in,model_out,test_model_in,test_model_out,validate_model_in,validate_model_out=get_input_output(targets)

In [22]:
#cluster中元素对应的index
max_arr=numpy.array([ [seg_max_min[seg][0]] for seg in targets])
min_arr=numpy.array([[seg_max_min[seg][1]] for seg in targets])
bias_arr=max_arr-min_arr#(路段0，路段1,...)
# bias_arr=numpy.array([[i] for i in bias_arr])
bias_arr=numpy.repeat(bias_arr,test_days*day_records_num,axis=0) #测试集天数*每天记录数
# bias_arr=bias_arr.reshape(bias_arr.shape[0],1,1)
min_arr=numpy.repeat(min_arr,test_days*day_records_num,axis=0)
# min_arr=min_arr.reshape(min_arr.shape[0],1,1)
all_pred=c.predict(test_model_in)
pred=all_pred*bias_arr+min_arr
gt=(test_model_out[0]*bias_arr+min_arr)
mae_tmp=numpy.absolute(pred-gt)
mape_tmp=mae_tmp/gt
print(numpy.mean(mae_tmp))
print(numpy.mean(mape_tmp))

32.849078598484844
0.3788567278325449


In [31]:
test_days

7

In [25]:
mape_tmp.shape

(7392, 1)

In [24]:
targets=[9, 277]
model_in,model_out,test_model_in,test_model_out,validate_model_in,validate_model_out=get_input_output(targets)

In [26]:
#cluster中元素对应的index
max_arr=numpy.array([ [seg_max_min[seg][0]] for seg in targets])
min_arr=numpy.array([[seg_max_min[seg][1]] for seg in targets])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,test_days*day_records_num,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,test_days*day_records_num,axis=0)
all_pred=c0.predict(test_model_in)
pred=all_pred*bias_arr+min_arr
gt=(test_model_out[0]*bias_arr+min_arr)
mae_0=numpy.absolute(pred-gt)
mape_0=mae_0/gt
print(numpy.mean(mae_0))
print(numpy.mean(mape_0))


10.996326906613175
0.09891909365306967


In [27]:
mape_0.shape

(2464, 1)

In [28]:
targets=[24,256,267,268]
model_in,model_out,test_model_in,test_model_out,validate_model_in,validate_model_out=get_input_output(targets)

In [29]:
#cluster中元素对应的index
max_arr=numpy.array([ [seg_max_min[seg][0]] for seg in targets])
min_arr=numpy.array([[seg_max_min[seg][1]] for seg in targets])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,test_days*day_records_num,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,test_days*day_records_num,axis=0)
all_pred=c1.predict(test_model_in)
pred=all_pred*bias_arr+min_arr
gt=(test_model_out[0]*bias_arr+min_arr)
mae_1=numpy.absolute(pred-gt)
mape_1=mae_1/gt
print(numpy.mean(mae_1))
print(numpy.mean(mape_1))

7.31511241094576
0.09107086929110805


In [30]:
numpy.mean(numpy.concatenate((mape_0,mape_1)))

0.0936869440784286

### log 2

In [8]:
seg_max_min=[
    [136.44, 69.0],
[58.14, 29.0],
# [68.03999999999999, 34.0],
[90.0, 31.0],
[134.46, 63.0],
[102.06, 51.0],
[136.44, 64.0]
]

In [4]:
# C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2\/test.npy
train=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2\/train.npy',allow_pickle=True).item()
test=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2\/test.npy',allow_pickle=True).item()
validate=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_2\/validation.npy',allow_pickle=True).item()
dw_train_0=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\/train_day_of_week.npy',)
dw_test_0=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\/test_day_of_week.npy',)
dw_validate_0=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\/validate_day_of_week.npy',)
ts=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\encoded_time_stamp.npy')

In [58]:
train_in_c=[]
train_in_d=[]
train_out_c=[]
test_in_c=[]
test_in_d=[]
test_out_c=[]
validate_in_c=[]
validate_in_d=[]
validate_out_c=[]
target_seg=[9,277]
target_seg_len=len(target_seg)
for seg in target_seg:#路段key
    train_in_c+=( train[seg][0])
    train_in_d+=( train[seg][1])
    train_out_c+=( train[seg][2])
    test_in_c+=(test[seg][0])
    test_in_d+=(test[seg][1])
    test_out_c+=(test[seg][2])
    validate_in_c+=( validate[seg][0])
    validate_in_d+=( validate[seg][1])
    validate_out_c+=( validate[seg][2])

In [59]:

##路段0+路段1+....
# dw_train=numpy.repeat(dw_train_0,target_seg_len)#路段数
# dw_test=numpy.repeat(dw_test_0,target_seg_len)
dw_validate=[]
for i in range(target_seg_len):
    dw_validate=numpy.concatenate((dw_validate,dw_validate_0))
dw_train=[]
for i in range(target_seg_len):
    dw_train=numpy.concatenate((dw_train,dw_train_0))
dw_test=[]
for i in range(target_seg_len):
    dw_test=numpy.concatenate((dw_test,dw_test_0))
ts_train_0=[]
for i in range(28):
    ts_train_0=numpy.concatenate((ts_train_0,ts[:176]))
ts_test_0=[]
for i in range(7):
    ts_test_0=numpy.concatenate((ts_test_0,ts[:176]))
ts_validate_0=[]
for i in range(7):
    ts_validate_0=numpy.concatenate((ts_validate_0,ts[:176]))
ts_train=[]
for i in range(target_seg_len):
    ts_train=numpy.concatenate((ts_train,ts_train_0))
ts_test=[]
for i in range(target_seg_len):
    ts_test=numpy.concatenate((ts_test,ts_test_0))
ts_validate=[]
for i in range(target_seg_len):
    ts_validate=numpy.concatenate((ts_validate,ts_validate_0))



In [60]:
train_in_c=numpy.array(train_in_c)
train_in_d=numpy.array(train_in_d)
train_out_c=numpy.array(train_out_c)

test_in_c=numpy.array(test_in_c)
test_in_d=numpy.array(test_in_d)
test_out_c=numpy.array(test_out_c)

validate_in_c=numpy.array(validate_in_c)
validate_in_d=numpy.array(validate_in_d)
validate_out_c=numpy.array(validate_out_c)

train_in_c=train_in_c.reshape(train_in_c.shape[0],4,1)
train_in_d=train_in_d.reshape(train_in_d.shape[0],4,1)
train_out_c=train_out_c.reshape(train_out_c.shape[0],1)

test_in_c=test_in_c.reshape(test_in_c.shape[0],4,1)
test_in_d=test_in_d.reshape(test_in_d.shape[0],4,1)
test_out_c=test_out_c.reshape(test_out_c.shape[0],1)

validate_in_c=validate_in_c.reshape(validate_in_c.shape[0],4,1)
validate_in_d=validate_in_d.reshape(validate_in_d.shape[0],4,1)
validate_out_c=validate_out_c.reshape(validate_out_c.shape[0],1)

ts_train=ts_train.reshape(ts_train.shape[0],1)
ts_test=ts_test.reshape(ts_test.shape[0],1)
ts_validate=ts_validate.reshape(ts_validate.shape[0],1)

dw_test=dw_test.reshape(dw_test.shape[0],1)
dw_train=dw_train.reshape(dw_train.shape[0],1)
dw_validate=dw_validate.reshape(dw_validate.shape[0],1)

ts_train=ts_train.reshape(ts_train.shape[0],1)
ts_test=ts_test.reshape(ts_test.shape[0],1)
ts_validate=ts_validate.reshape(ts_validate.shape[0],1)

dw_test=dw_test.reshape(dw_test.shape[0],1)
dw_train=dw_train.reshape(dw_train.shape[0],1)
dw_validate=dw_validate.reshape(dw_validate.shape[0],1)

model_in=[train_in_c,train_in_d,dw_train,ts_train]
model_out=[train_out_c]
test_model_in=[test_in_c,test_in_d,dw_test,ts_test]
test_model_out=[test_out_c]
validate_model_in=[validate_in_c,validate_in_d,dw_validate,ts_validate]
validate_model_out=[validate_out_c]

In [36]:
#cluster中元素对应的index
valid_index=[i for i in range(6)]
max_arr=numpy.array([ [seg_max_min[i][0]] for i in valid_index])
min_arr=numpy.array([[seg_max_min[i][1]] for i in valid_index])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,7*176,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,7*176,axis=0)

In [39]:
all_pred=c.predict(test_model_in)
pred=all_pred*bias_arr+min_arr
gt=(numpy.array(test_model_out[0])*bias_arr+min_arr)
tmp=numpy.absolute(pred-gt)/gt
print(numpy.mean(tmp))
# sum_mape=numpy.concatenate((sum_mape,tmp),axis=0)

0.09170164807504778


In [51]:
#cluster中元素对应的index
valid_index=[1,2,3,4]
max_arr=numpy.array([ [seg_max_min[i][0]] for i in valid_index])
min_arr=numpy.array([[seg_max_min[i][1]] for i in valid_index])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,7*176,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,7*176,axis=0)

In [57]:
pred=c1.predict(test_model_in)
pred=pred*bias_arr+min_arr
gt=(numpy.array(test_model_out[0])*bias_arr+min_arr)
tmp=numpy.absolute(pred-gt)/gt
print(numpy.mean(tmp))
sum_mape=tmp
# sum_mape=numpy.concatenate((sum_mape,tmp))

0.08745809322326148


In [61]:
#cluster中元素对应的index
valid_index=[0,5]
max_arr=numpy.array([ [seg_max_min[i][0]] for i in valid_index])
min_arr=numpy.array([[seg_max_min[i][1]] for i in valid_index])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,7*176,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,7*176,axis=0)

In [62]:
pred=c0.predict(test_model_in)
pred=pred*bias_arr+min_arr
gt=(numpy.array(test_model_out[0])*bias_arr+min_arr)
tmp=numpy.absolute(pred-gt)/gt
print(numpy.mean(tmp))
sum_mape=numpy.concatenate((sum_mape,tmp))
# sum_mape=tmp

0.0881868456389677


In [63]:
numpy.mean(sum_mape)

0.08770101069516356